# Computing the Bathymetry between stations

In this section, we compute the bathymetry between stations. This will be useful to then plot vertical section along some stations (i.e., a transect).

The bathymetry was downloaded from the GEBCO site (https://download.gebco.net/) in a netCDF file.

In [2]:
#Imports
import sys
import numpy as np
import pandas as pd
import netCDF4
from shapely.geometry import Point, LineString, MultiPoint

#Import path to custom functions
sys.path.append('C:/Users/pauab/Universidad/aProgramacion/TFM/Notebooks_Pau/my_functions/')
#Import custom functions
from read_CTD import read_CTD    #To read CTD files
from out_outliers import out_outliers     #To replace outliers

#Import path to custom functions
sys.path.append('C:/Users/pauab/Universidad/aProgramacion/TFM/Notebooks_Pau/my_functions/distance/')
from distances import *

#To turn off warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
#Data reading and outlier replacement
data_dirs = ['C:/Users/pauab/Universidad/Data/TFM/CTD_recalc/']
data=read_CTD(data_dirs)

for ncast in data.keys():
    for variable in data[ncast].columns:
        out_outliers(data, ncast, variable)
    data[ncast]['pressure']=data[ncast].index
    data[ncast].index = np.arange(0, len(data[ncast].index))
    
#Dict with all transsects
transsects = {'transsect_0':['15','12','13','14'],
              'transsect_1':['19','18','17','16'],
              'transsect_2':['24','23','22','21'],
              'transsect_3':['26','27','28','29'],
              'transsect_4':['31','32','33','34'],
              'transsect_5':['38','37','36','35'],
              'transsect_6':['43','44','45','46']}

In [6]:
# Get bathymetry for the selected section
# Get GEBCO nc file. This is the file for around iberian peninsula
gebco = 'C:/Users/pauab/Universidad/Data/TFM/Bathymetry/GEBCO_16_Feb_2022_8d3ca050bc32/gebco_2021_n44.857177734375_s34.991455078125_w-11.97509765625_e-4.37255859375.nc'

with netCDF4.Dataset(gebco) as netcdf:
#     print(netcdf.variables)
    x = np.array(netcdf.variables['lon'][:])
    y = np.array(netcdf.variables['lat'][:])
    z = np.squeeze(np.array(netcdf.variables['elevation'][:]))
#     projection = netcdf.variables['albers_conical_equal_area'].spatial_ref


#Compute the bathymetry profile
bathy_transsects = {}
xoffset=False         #xoffset in km. False means no offset
#Iterate over the transsects where you want to compute the bathymetry
for transsect in transsects.keys():
    profile_df = pd.DataFrame()
    profile_full = pd.DataFrame() # If stations align well might be useful
    station_index = [0]
    
    #Get the lat and lon of the transsect stations
    station_lats = []
    station_longs = []
    for ncast in transsects[transsect]:
        station_lats.append(data[ncast]['latitude'].iloc[-1])
        station_longs.append(data[ncast]['longitude'].iloc[-1])
        
    #variables needed
    sect_start_lon = station_longs[0]
    sect_start_lat = station_lats[0]

    # If we start the section before the first station, we also want bathy then
    if xoffset:
        start_profile = Point(sect_start_lon, sect_start_lat)    # Calc first segment points
        end_profile = Point(station_longs[0], station_lats[0])

        # Get first segment into dataframe
        profile_df = pd.DataFrame(get_profile(start_profile, end_profile, resolution=1000))

        # Store station indexes
        station_index.append(profile_df.index[-1])
        tmp_distance = profile_df['distance'].iloc[-1] #experimental
        #print(tmp_distance)

    # Iterate over CTD stations
    for i in range(len(station_lats)):

        # All stations 
        if i < len(station_lats)-1:
            start_profile = Point(station_longs[i], station_lats[i])   # Cacl segment points
            end_profile = Point(station_longs[i+1], station_lats[i+1])

            # In case no xoffset, then we need to populate dataframe first
            if profile_df.empty:
                profile_df = pd.DataFrame(get_profile(start_profile, end_profile, resolution=1000))
                station_index.append(profile_df.index[-1])
                tmp_distance = profile_df['distance'].iloc[-1] 

            else:
                profile_temp = pd.DataFrame(get_profile(start_profile, end_profile, resolution=1000))

                # Temp df to perform cumulative sum of distances. Incremental addition
                profile_temp['distance'] = profile_temp['distance'] + tmp_distance # experimental
                profile_df = pd.concat([profile_df, profile_temp])
                station_index.append(profile_df.index[-1])
                tmp_distance = profile_df['distance'].iloc[-1] # Store max distance for next iteration

        # When at last station, we want bathy to be extended by xoffset also
        elif i == len(station_lats)-1:
            start_profile = Point(station_longs[i], station_lats[i])
            end_profile = Point(station_longs[i] - xoffset, station_lats[i])

            profile_temp = pd.DataFrame(get_profile(start_profile, end_profile, resolution=1000))
            profile_temp['distance'] = profile_temp['distance'] + tmp_distance 
            profile_df = pd.concat([profile_df, profile_temp])
            station_index.append(profile_df.index[-1])

    # Reset indexes in dataframe and add as column
    profile_df.reset_index(drop=True, inplace=True)
    profile_df['comp_dist'] = profile_df.index
    bathy_transsects[transsect] = profile_df

We created the function bathymatry_transsects.py to simplify the previous code. The function computes the bathymetry field between the stations of a given transect.

It returns a dict where the bathymetry field (value) is stored according to the transsect number (key)